# Miniature/spontaneous postsynaptic currents: Part 2

In part 1 we covered how to extract the mEPSCs or sEPSCs. In this chapter we will dive into a more rigorous analysis of PSCs as well as look at how the shape of a PSC effects their integration.

In [ ]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Select, Checkbox
from bokeh.plotting import figure
from scipy import stats

output_notebook()

In [2]:
url_path = "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/"
pv = pd.read_csv(url_path + "mini_data.csv")

## Relationships between variables

In [6]:
fig = figure(height=250, width=400)
columns = [
    "Est Tau (ms)",
    "Rise Time (ms)",
    "Amplitude (pA)",
    "Rise Rate (pA/ms)",
    "IEI (ms)",
]

hist_dict = {}
for i in columns:
    data = pv[i]
    min_val = min(data)
    max_val = max(data)
    padding = (max_val - min_val) * 0.1  # Add 10% padding
    grid_min = min_val - padding
    grid_max = max_val + padding
    grid_min = max(grid_min, 0)
    positions = np.linspace(grid_min, grid_max, num=124)
    kernel = stats.gaussian_kde(data)
    y = kernel(positions)
    hist_dict[f"{i}_x"] = positions
    hist_dict[f"{i}_y"] = y

hist_dict["y"] = hist_dict["Amplitude (pA)_y"]
hist_dict["x"] = hist_dict["Amplitude (pA)_x"]
source = ColumnDataSource(hist_dict)

line = fig.line(x="x", y="y", source=source, line_color="black")

menu = Select(title="Variables", value=columns[0], options=columns)

callback = CustomJS(
    args=dict(
        source=source,
        menu=menu,
    ),
    code="""
    console.log(menu.value);
    source.data.y = source.data[`${menu.value}_y`];
    source.data.x = source.data[`${menu.value}_x`];
    source.change.emit();
""",
)

menu.js_on_change("value", callback)

show(column(row(menu), row(fig)))

In [ ]:
scaled_events = events - events.max(axis=1, keepdims=True)
scaled_events /= abs(scaled_events.min(axis=1, keepdims=True))
source1 = ColumnDataSource(
    {"x": list(x), "events": list(events), "scaled_events": list(scaled_events)}
)
source2 = ColumnDataSource(
    {
        "x": np.arange(max_event_length) / 10,
        "avg_event": events.mean(axis=0),
        "avg_scaled": scaled_events.mean(axis=0),
    }
)
fig1 = figure(title="Events", height=250, width=400, output_backend="webgl")
fig1.multi_line("x", "events", source=source1, alpha=0.2, color="black")
fig1.line("x", "avg_event", source=source2, color="orange")
fig2 = figure(title="Scaled events", height=250, width=400, output_backend="webgl")
fig2.multi_line("x", "scaled_events", source=source1, alpha=0.2, color="black")
fig2.line("x", "avg_scaled", source=source2, color="orange")
show(row(fig1, fig2))